In [1]:
import re
import pandas as pd
import numpy as np
import nltk
import collections, re
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords 
from sklearn.preprocessing import LabelBinarizer
from collections import Counter
import difflib as dl
import string
from sklearn.model_selection import train_test_split
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tommyjiang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
stop_words = set(stopwords.words('english'))
def remove_stop(astring):
    l_string=[]
    k=astring.split(" ")
    for s in k:
        if s not in stop_words:
            l_string.append(s)
    if len(l_string)==0:
        return "None"
    else:
        return " ".join(l_string)

In [3]:
df=pd.read_csv("fake_job_postings.csv") #you need to change the path
salary=pd.read_excel("state_M2018_dl.xlsx") #you need to change the path
salary_us=pd.read_excel("national_M2018_dl.xlsx") #you need to change the path
semi_df=pd.read_csv("monster_com-job_sample.csv") #you need to change the path

In [4]:
#Missing data for original dataframe
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Missing Percent'])
missing_data['Missing Percent'] = missing_data['Missing Percent'].apply(lambda x: x * 100)
missing_data.loc[missing_data['Missing Percent'] >=0]

,Total,Missing Percent
salary_range,15012,83.959732
department,11547,64.580537
required_education,8105,45.329978
benefits,7210,40.324385
required_experience,7050,39.429530
function,6455,36.101790
industry,4903,27.421700
employment_type,3471,19.412752
company_profile,3308,18.501119
requirements,2695,15.072707


In [5]:
#Data Clean - title (for semi supervised use)
def clean_string(subject):
    clean_tokens = re.findall(r"(?i)\b[a-z]+\b", subject)
    clean_s = ' '.join(clean_tokens)
    return clean_s

In [6]:
# Data Clean - location
location=df[['job_id','location','fraudulent']].fillna("None,None,None")
location['location_clean']=location['location'].apply(lambda row: str(row).split(","))
correct_location=location[location['location_clean'].str.len()==3]
split_location= pd.DataFrame(correct_location['location_clean'].values.tolist(), index= correct_location.index)
combined_clean_location=pd.concat([correct_location,split_location],axis=1)
combined_clean_location.rename(columns={0:'Country',1:"State",2:"City"},inplace=True)
combined_clean_location['Country']=combined_clean_location['Country'].str.replace('[^a-zA-Z,]','')
combined_clean_location['State']=combined_clean_location['State'].str.replace('[^a-zA-Z,]', '')
combined_clean_location['City']=combined_clean_location['City'].str.replace('\d+', '')
combined_clean_location['City']=combined_clean_location['City'].str.replace(':', '')
combined_clean_location['City']=combined_clean_location['City'].str.replace('-', '')
combined_clean_location.replace(to_replace=" ", value="None",inplace=True)
combined_clean_location.replace(to_replace="  ", value="None",inplace=True)
combined_clean_location.replace(to_replace="", value="None",inplace=True)
Incorrect_location=location[location['location_clean'].str.len()!=3]
Incorrec_location_1=Incorrect_location[Incorrect_location['location_clean'].map(len)==1]
Incorrec_location_2=Incorrec_location_1.copy()
Incorrec_location_2['Country']=Incorrect_location["location"]
Incorrec_location_2['State']="None"
Incorrec_location_2['City']="None"
Incorrec_location_3=Incorrect_location[Incorrect_location['location_clean'].map(len)!=1]
Incorrec_location_4=Incorrec_location_3.copy()
Incorrec_location_4.loc[:,"Country"]=Incorrec_location_3["location_clean"].map(lambda x:x[0])
Incorrec_location_4.loc[:,"State"]=Incorrec_location_3["location_clean"].map(lambda x:x[1])
Incorrec_location_4.loc[:,"City"]=Incorrec_location_3["location_clean"].map(lambda x:x[2])
Incorrec_location_4.replace(to_replace=" ", value="None",inplace=True)
Incorrec_location_4.replace(to_replace="", value="None",inplace=True)
Incorrec_location_4
clean_location=pd.concat([combined_clean_location,Incorrec_location_4,Incorrec_location_2],axis=0)
clean_location_output=clean_location[['job_id','Country','State']].copy()
clean_location_output.head()

,job_id,Country,State
0,1,US,NY
1,2,NZ,None
2,3,US,IA
3,4,US,DC
4,5,US,FL


In [7]:
#Data Clean-Department
department=df[['job_id','department','fraudulent']].copy()
department=department.fillna("None")
department['department']=department['department'].str.lower()
regex = re.compile('[@_!#$%^&*()<>?/\|}{~:]')
department['check']=department['department'].apply(lambda x: regex.search(x)!=None)
department_clean=department.copy()
department_clean['clean_department']=department['department'].apply(lambda x: clean_string(x))
department_fraud_ranking=department_clean.groupby("clean_department").agg({'fraudulent':'sum'}).sort_values(by='fraudulent',ascending=False).reset_index()
department_fraud_ranking
clean_department_output=department_clean[['job_id','clean_department']].copy()
clean_department_output["clean_department"]=clean_department_output["clean_department"]!="None"
clean_department_output["clean_department"]=clean_department_output["clean_department"]*1
clean_department_output["Special_Char"]=department['check']*1
clean_department_output.head()

,job_id,clean_department,Special_Char
0,1,1,0
1,2,1,0
2,3,1,0
3,4,1,0
4,5,1,0


In [8]:
#Data clean-telecommuting, has company_logo, has_questions
clean_tlq_output=df[['job_id','telecommuting','has_company_logo','has_questions']]
clean_tlq_output.head()

,job_id,telecommuting,has_company_logo,has_questions
0,1,0,1,0
1,2,0,1,0
2,3,0,1,0
3,4,0,1,0
4,5,0,1,1


In [9]:
#Data clean-employment type
clean_employment_output=df[['job_id','employment_type']].fillna("None")
clean_employment_output.head()

,job_id,employment_type
0,1,Other
1,2,Full-time
2,3,None
3,4,Full-time
4,5,Full-time


In [10]:
#Data clean - experience
experience=df[['job_id','required_experience','fraudulent']]
experience=experience.fillna("None")
experience["required_experience"]=experience["required_experience"].replace("Not Applicable", "None") 
clean_experience_output=experience[["job_id","required_experience"]]
clean_experience_output.head()

,job_id,required_experience
0,1,Internship
1,2,None
2,3,None
3,4,Mid-Senior level
4,5,Mid-Senior level


In [11]:
#Data clean - required eductation
education=df[['job_id','required_education','fraudulent']].copy()
education=education.fillna("None")
education["required_education"]=education["required_education"].replace("Vocational - Degree", "Vocational") 
education["required_education"]=education["required_education"].replace("Vocational - HS Diploma", "Vocational") 
education["required_education"]=education["required_education"].replace("Unspecified", "None") 
education["required_education"]=education["required_education"].replace("Some High School Coursework", "High School") 
education["required_education"]=education["required_education"].replace("High School or equivalent", "High School") 
education["required_education"]=education["required_education"].replace("Certification", "Prof & Cert") 
education["required_education"]=education["required_education"].replace("Professional", "Prof & Cert") 
education["required_education"]=education["required_education"].replace("Bachelor's Degree", "Bachelor & Assoc") 
education["required_education"]=education["required_education"].replace("Associate Degree", "Bachelor & Assoc") 
education["required_education"]=education["required_education"].replace("Doctorate", "Master & Doctorate") 
education["required_education"]=education["required_education"].replace("Master's Degree", "Master & Doctorate") 
education["required_education"]=education["required_education"].replace("Some College Coursework Completed", "Bachelor & Assoc") 
clean_education_output=education[["job_id","required_education"]]
clean_education_output.head()

,job_id,required_education
0,1,None
1,2,None
2,3,None
3,4,Bachelor & Assoc
4,5,Bachelor & Assoc


In [12]:
#data clean - industry
industry=df[['job_id','industry','fraudulent']].copy()
industry=industry.fillna("None")
industry["industry"]=industry["industry"].apply(lambda x: clean_string(x))
industry["industry"]=industry["industry"].str.lower()
industry["industry"]=industry["industry"].apply(lambda x: remove_stop(x))
clean_industry_output=industry[["job_id","industry"]]
clean_industry_output.head()

,job_id,industry
0,1,none
1,2,marketing advertising
2,3,none
3,4,computer software
4,5,hospital health care


In [13]:
function=df[['job_id','function','fraudulent']].copy()
function=function.fillna("None")
function["function"]=function["function"].apply(lambda x: clean_string(x))
function["function"]=function["function"].str.lower()
clean_function_output=function[["job_id","function"]]
clean_function_output.head()

,job_id,function
0,1,marketing
1,2,customer service
2,3,none
3,4,sales
4,5,health care provider


In [14]:
#merge all cleaned data together except text columns and salary
s1=clean_location_output
s2=pd.merge(clean_department_output,clean_tlq_output,how='inner', on="job_id")
s3=pd.merge(clean_experience_output,clean_education_output,how='inner', on="job_id")
s4=pd.merge(clean_industry_output,clean_function_output,how='inner', on="job_id")
s5=pd.merge(s1,s2,how='inner', on="job_id")
s6=pd.merge(s5,clean_employment_output,how='inner', on="job_id")
s7=pd.merge(s3,s4,how='inner', on="job_id")
s8=pd.merge(s6,s7,how='inner', on="job_id")
s9=df[["job_id","fraudulent"]]
s10=pd.merge(s8,s9,how='inner', on="job_id")
cleaned_df=s10
cleaned_df.head()

,job_id,Country,State,clean_department,Special_Char,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,US,NY,1,0,0,1,0,Other,Internship,None,none,marketing,0
1,2,NZ,None,1,0,0,1,0,Full-time,None,None,marketing advertising,customer service,0
2,3,US,IA,1,0,0,1,0,None,None,None,none,none,0
3,4,US,DC,1,0,0,1,0,Full-time,Mid-Senior level,Bachelor & Assoc,computer software,sales,0
4,5,US,FL,1,0,0,1,1,Full-time,Mid-Senior level,Bachelor & Assoc,hospital health care,health care provider,0


In [15]:
# Clean salary
salarydf = df[['job_id', 'fraudulent','salary_range']].dropna()
salarydf[['Min','Max']] = salarydf['salary_range'].str.split("-",expand=True)

# Single Value (Min=Max)
salarydf.loc[salarydf['Max'].isna(), 'Max'] = salarydf['Min']

# Remove Errors
salarydf['Min'] = pd.to_numeric(salarydf['Min'], errors='coerce')
salarydf['Max'] = pd.to_numeric(salarydf['Max'], errors='coerce')
salarydf.loc[salarydf['Min'].isna(), 'Max'] = np.nan
salarydf.loc[salarydf['Max'].isna(), 'Min'] = np.nan

# Zero to Missing Value
salarydf[['Min','Max']] = salarydf[['Min','Max']].replace(0, np.nan)

# Correct Minor Errors
salarydf[['Min', 'Max']] = salarydf[['Min', 'Max']].replace(1, np.nan)

# Convert Unit

# Check if range is too large
salarydf['Ambiguous Range'] = 0
salarydf.loc[salarydf['Max']/salarydf['Min']>3, 'Ambiguous Range'] = 1
salarydf_0 = salarydf[salarydf['Ambiguous Range']==0]
salarydf_1 = salarydf[salarydf['Ambiguous Range']==1]

# Assume <$300 range is in hourly rate
salarydf_0.loc[salarydf_0['Min']<300, 'Max'] = salarydf_0['Max']*40*52
salarydf_0.loc[salarydf_0['Min']<300, 'Min'] = salarydf_0['Min']*40*52

# Assume <$1000 range is in weekly rate
salarydf_0.loc[salarydf_0['Min']<1000, 'Max'] = salarydf_0['Max']*52
salarydf_0.loc[salarydf_0['Min']<1000, 'Min'] = salarydf_0['Min']*52

# Assume <$10000 range is in monthly rate
salarydf_0.loc[salarydf_0['Min']<10000, 'Max'] = salarydf_0['Max']*12
salarydf_0.loc[salarydf_0['Min']<10000, 'Min'] = salarydf_0['Min']*12

# Median (for comparison with external data?)
salarydf_0['Median'] = (salarydf_0['Min']+salarydf_0['Max'])/2

salarydf_0 = salarydf_0.sort_values(by=['Min'], ascending=True)

salarydf_0 = salarydf_0.dropna()

salary_clean = df[['job_id', 'fraudulent']]
salary_clean = salary_clean.merge(salarydf_0, how='outer').merge(salarydf_1, how='outer').drop(columns='salary_range')
salary_clean = salary_clean.sort_values(by=['Ambiguous Range'], ascending=True)
salary_clean['Missing'] = 0
salary_clean.loc[salary_clean['Min'].isna(), 'Missing'] = 1
salary_clean.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,job_id,fraudulent,Min,Max,Ambiguous Range,Median,Missing
6,7,0,20000.0,28000.0,0.0,24000.0,0
12025,12026,0,90000.0,130000.0,0.0,110000.0,0
12027,12028,0,18000.0,20000.0,0.0,19000.0,0
12042,12043,0,200000.0,300000.0,0.0,250000.0,0
12053,12054,0,18000.0,20000.0,0.0,19000.0,0


In [16]:
#Salary matching
salary_matching = salary[['ST', 'OCC_TITLE', 'OCC_GROUP', 'A_MEDIAN']]
salary_matching = salary_matching.loc[salary_matching['OCC_GROUP']=='detailed']
salary_matching = salary_matching.drop(columns='OCC_GROUP')

def CleanDept(x):
    x = x.lower()
    x = x.replace('and', ',')
    x = x.replace('managers', '')
    return x

salary_matching['OCC_TITLE'] = salary_matching['OCC_TITLE'].apply(lambda x: CleanDept(x))
salary_matching['dept_token'] = salary_matching['OCC_TITLE'].apply(lambda row: row.split(','))

salary_matching['state-title'] = salary_matching['ST']+'-'+salary_matching['OCC_TITLE']
salary_matching_us = salary[['OCC_TITLE', 'OCC_GROUP', 'A_MEDIAN']]
salary_matching_us = salary_matching_us.loc[salary_matching_us['OCC_GROUP']=='detailed']
salary_matching_us = salary_matching_us.drop(columns='OCC_GROUP')

salary_matching_us['OCC_TITLE'] = salary_matching_us['OCC_TITLE'].apply(lambda x: CleanDept(x))
salary_matching_us['dept_token'] = salary_matching_us['OCC_TITLE'].apply(lambda row: row.split(','))

salary_matching_us['state-title'] ='US-'+salary_matching_us['OCC_TITLE']
salary_dict = pd.Series(salary_matching.A_MEDIAN.values,index=salary_matching['state-title']).to_dict()
salary_dict_us = pd.Series(salary_matching_us.A_MEDIAN.values,index=salary_matching_us['state-title']).to_dict()
salary_dict.update(salary_dict_us)


occ_list = []
for i in salary_matching['dept_token'].values:
    occ_list.extend(i)

occ_list = list(set(occ_list))

def GetMatches(x):
    return dl.get_close_matches(x, occ_list, n=1, cutoff=0.1)

dept=df[['job_id','department','fraudulent']].dropna()
dept['department']=dept['department'].str.lower()
dept['matches'] = dept['department'].apply(lambda x: GetMatches(x))
## Use location table generated above
dept_combined = dept.merge(combined_clean_location[['job_id', 'State', 'Country']], how='outer').merge(salary_clean[['job_id', 'Median']], how='outer')
match_dict = pd.Series(salary_matching.dept_token.values,index=salary_matching.OCC_TITLE).to_dict()

def LtoS(x):
    if x is not np.nan:
        if len(x)>0:
            return x[0]
    else:
        return np.nan

def CallDept(x):
    for i in range(len(list(match_dict.values()))):
        if x in list(match_dict.values())[i]:
            return list(match_dict.keys())[i]

dept_combined['match_clean'] = dept_combined['matches'].apply(lambda x: LtoS(x))
dept_combined['OCC_TITLE'] = dept_combined['match_clean'].apply(lambda x: CallDept(x))
dept_toimpute = dept_combined.loc[(dept_combined['Country']=='US')&(dept_combined['Median'].isna())&(dept_combined['match_clean'].notna())&(dept_combined['State'].notna())]
dept_toimpute['State'] = dept_toimpute['State'].replace(' ', 'US')
dept_toimpute['State'] = dept_toimpute['State'].replace('', 'US')
dept_toimpute['State'] = dept_toimpute['State'].apply(lambda x: x.strip())

def MatchState(x):
    key = str(dept_toimpute['State']+'-'+x)
    if key in list(salary_matching['state-title'].unique()):
        return key
    else:
        key = str('US-'+x)
        return key

def Impute(x):
    return salary_dict[x]

dept_toimpute['state-title'] = dept_toimpute['OCC_TITLE'].apply(lambda x: MatchState(x))
dept_toimpute['Median'] = dept_toimpute['state-title'].apply(lambda x: Impute(x))
dept_exist = dept_combined.loc[dept_combined['Median'].notna()]
dept_imputed = pd.concat([dept_exist, dept_toimpute.iloc[:,:-1]])
dept_imputed = dept_imputed.drop(columns=['matches', 'match_clean'])
def SalaryStat(x):
    try:
        key = str(dept_toimpute['State']+'-'+x)
        if key in list(salary_matching['state-title'].unique()):
            return salary_dict[key]
        else:
            key = str('US-'+x)
            return salary_dict[key]
    except:
        pass

dept_imputed['salary_stat'] = dept_imputed['OCC_TITLE'].apply(lambda x: SalaryStat(x))
dept_imputed.loc[dept_imputed['Country']!='US', 'salary_stat'] = np.nan
dept_imputed['Median'] = pd.to_numeric(dept_imputed['Median'], errors='coerce')
dept_imputed['salary_stat'] = pd.to_numeric(dept_imputed['salary_stat'], errors='coerce')
dept_imputed['missing_data'] = 0
dept_imputed['ambiguous_salary'] = 0
dept_imputed.loc[dept_imputed['salary_stat']==np.nan, 'missing_data'] = 1
dept_imputed.loc[(dept_imputed['missing_data']==0)&(dept_imputed['Median']/dept_imputed['salary_stat']<0.5), 'ambiguous_salary'] = 1
dept_imputed.loc[(dept_imputed['missing_data']==0)&(dept_imputed['Median']/dept_imputed['salary_stat']>2), 'ambiguous_salary'] = 1
salary_combined = salary_clean[['job_id', 'fraudulent', 'Ambiguous Range', 'Missing']].merge(dept_imputed[['job_id', 'missing_data', 'ambiguous_salary']], how='outer')
salary_combined = salary_combined.rename(columns={'missing_data':'missing_stat', 'Missing': 'missing_salary'})
salary_combined = salary_combined.fillna(1)
salary_combined.loc[salary_combined['missing_stat']==1, 'ambiguous_salary'] = 1
salary_combined = salary_combined.drop_duplicates(subset='job_id')
salary_combined.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

,job_id,fraudulent,Ambiguous Range,missing_salary,missing_stat,ambiguous_salary
0,7,0,0.0,0,0.0,0.0
1,12026,0,0.0,0,0.0,0.0
2,12028,0,0.0,0,0.0,0.0
3,12043,0,0.0,0,0.0,0.0
4,12054,0,0.0,0,0.0,0.0


In [17]:
#clean benifit
benefit = df[['job_id', 'benefits', 'fraudulent']].copy()
benefit.loc[benefit['benefits']=='#NAME?', 'benefits'] = np.nan
benefit.loc[benefit['benefits']=='---', 'benefits'] = np.nan
benefit.loc[benefit['benefits']=='See job description', 'benefits'] = np.nan

benefit['missing'] = 0
benefit.loc[benefit['benefits'].isna(), 'missing'] = 1

In [18]:
def clean_benefit_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
     and remove words containing numbers.'''
    text = re.sub(r"(\w)([A-Z])", r"\1 \2", text)
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

benefit2 = benefit.dropna()
tokenizer2 = RegexpTokenizer(r'[a-zA-Z]{1,}')
benefit2['benefits_cleaned'] = benefit2['benefits'].apply(lambda x: clean_benefit_text(x))
benefit2['unigram'] = benefit2['benefits_cleaned'].apply(lambda row: tokenizer2.tokenize(row))
benefit2['bigram'] = benefit2['benefits_cleaned'].apply(lambda row: list(ngrams(tokenizer2.tokenize(row),2)))
# 3 Major Benefits = Medical, Life, Retirement

med_unigram = ['medical', 'health']
life_unigram = ['life']
ret_unigram = ['401', '401k', 'retirement']

def BenefitDetect(unigram):
    num_benefits = 0

    if any(i.lower() in unigram for i in med_unigram):
        num_benefits +=1
    if any(i.lower() in unigram for i in life_unigram):
        num_benefits +=1
    if any(i.lower() in unigram for i in ret_unigram):
        num_benefits +=1
    if num_benefits >=1:
        return 1
    else:
        return 0

benefit2['major_benefits'] = benefit2['unigram'].apply(lambda x: BenefitDetect(x))
benefit_clean = df[['job_id', 'fraudulent']].merge(benefit2[['job_id', 'fraudulent', 'missing', 'major_benefits']], how='outer')
benefit_clean['missing'] = benefit_clean['missing'].fillna(1)
benefit_clean['major_benefits'] = benefit_clean['major_benefits'].fillna(0)
benefit_clean.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

,job_id,fraudulent,missing,major_benefits
0,1,0,1.0,0.0
1,2,0,0.0,0.0
2,3,0,1.0,0.0
3,4,0,0.0,0.0
4,5,0,0.0,0.0


In [19]:
#Combine All
all_together = cleaned_df.copy()
all_together = all_together.iloc[:,:-1].merge(salary_combined, how='inner').merge(benefit_clean, how='inner')
c_data=pd.get_dummies(all_together)
c_data_selection=c_data.drop(['job_id','fraudulent'], axis=1)
c_data_selection.head()

,clean_department,Special_Char,telecommuting,has_company_logo,has_questions,Ambiguous Range,missing_salary,missing_stat,ambiguous_salary,missing,...,function_public relations,function_purchasing,function_quality assurance,function_research,function_sales,function_science,function_strategy planning,function_supply chain,function_training,function_writing editing
0,1,0,0,1,0,1.0,1,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,1.0,1,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,1.0,1,1.0,1.0,1.0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,1.0,1,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
4,1,0,0,1,1,1.0,1,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
train_indices, test_indices, y_train, y_test = train_test_split(np.arange(len(df)), 
c_data['fraudulent'],stratify=c_data['fraudulent'],test_size=0.3,random_state=0)
c_data_selection_train = c_data_selection.iloc[train_indices]
c_data_selection_test = c_data_selection.iloc[test_indices]

In [21]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state=0,max_depth=20,n_estimators=100)
classifier.fit(c_data_selection_train, y_train)
feat_labels=list(c_data_selection_train.columns.values)
from sklearn.feature_selection import SelectFromModel
sfm = SelectFromModel(classifier, threshold=0.005)
sfm.fit(c_data_selection_train, y_train)
columns=[]
for feature_list_index in sfm.get_support(indices=True):
    columns.append(feat_labels[feature_list_index])

In [22]:
non_text=c_data[columns]

In [23]:
non_text['job_id']=c_data['job_id']
non_text['fraudulent']=c_data['fraudulent']
non_text.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Special_Char,telecommuting,has_company_logo,has_questions,Ambiguous Range,missing_salary,missing_stat,ambiguous_salary,missing,major_benefits,...,function_accounting auditing,function_administrative,function_customer service,function_engineering,function_information technology,function_none,function_other,function_sales,job_id,fraudulent
0,0,0,1,0,1.0,1,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,1,0,1.0,1,1.0,1.0,0.0,0.0,...,0,0,1,0,0,0,0,0,2,0
2,0,0,1,0,1.0,1,1.0,1.0,1.0,0.0,...,0,0,0,0,0,1,0,0,3,0
3,0,0,1,0,1.0,1,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,4,0
4,0,0,1,1,1.0,1,1.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,5,0


In [24]:
#Clean text data
import re
from nltk.corpus import stopwords
BAD_SYMBOLS_RE = re.compile('[^a-z ]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text) 
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    return text

prof_des=df[["job_id","title","company_profile","description"]]
prof_des['title']=prof_des['title'].astype(str)
prof_des['company_profile']=prof_des['company_profile'].astype(str)
prof_des['description']=prof_des['description'].astype(str)
prof_des['title'] = prof_des['title'].apply(clean_text)
prof_des['company_profile'] = prof_des['company_profile'].apply(clean_text)
prof_des['description'] = prof_des['description'].apply(clean_text)
prof_des=prof_des.replace('nan', np.nan).fillna("Not provided")
prof_des=prof_des.fillna("Not provided")
prof_des.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

,job_id,title,company_profile,description
0,1,marketing intern,food weve created groundbreaking awardwinning ...,food fastgrowing james beard awardwinning onli...
1,2,customer service cloud video production,seconds worlds cloud video production service ...,organised focused vibrant awesomedo passion cu...
2,3,commissioning machinery assistant cma,valor services provides workforce solutions me...,client located houston actively seeking experi...
3,4,account executive washington dc,passion improving quality life geography heart...,company esri environmental systems research in...
4,5,bill review manager,spotsource solutions llc global human capital ...,job title itemization review managerlocation f...


In [33]:
final_clean_data=pd.merge(prof_des,non_text,on="job_id",how="inner")
final_clean_data2=pd.merge(prof_des,c_data,on="job_id",how="inner")

In [34]:
final_clean_data.at[937,"description"]="Not provided"
final_clean_data.at[937,"description"]
final_clean_data2.at[937,"description"]="Not provided"
final_clean_data2.at[937,"description"]

'Not provided'

In [35]:
final_clean_data.to_csv("clean_data.csv",index=False)
final_clean_data2.to_csv("clean_data2.csv",index=False)

In [28]:
m_df=pd.read_csv("monster_com-job_sample.csv")
monster_df=m_df[["job_title","job_description"]]

In [29]:
monster_df.head()

,job_title,job_description
0,IT Support Technician Job in Madison,TeamSoft is seeing an IT Support Specialist to...
1,Business Reporter/Editor Job in Madison,The Wisconsin State Journal is seeking a flexi...
2,Johnson & Johnson Family of Companies Job Appl...,Report this job About the Job DePuy Synthes Co...
3,Engineer - Quality Job in Dixon,Why Join Altec? If you’re considering a career...
4,Shift Supervisor - Part-Time Job in Camphill,Position ID# 76162 # Positions 1 State CT C...


In [30]:
monster_df['job_title']=monster_df['job_title'].astype(str)
monster_df['job_description']=monster_df['job_description'].astype(str)
monster_df['job_title'] = monster_df['job_title'].apply(clean_text)
monster_df['job_description'] = monster_df['job_description'].apply(clean_text)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [31]:
monster_df.head()

,job_title,job_description
0,support technician job madison,teamsoft seeing support specialist join client...
1,business reportereditor job madison,wisconsin state journal seeking flexible motiv...
2,johnson johnson family companies job applicati...,report job job depuy synthes companies member ...
3,engineer quality job dixon,join altec youre considering career altec inc ...
4,shift supervisor parttime job camphill,position id positions state ct city fairfield ...


In [32]:
monster_df.to_csv("clean_monster_data.csv",index=False)